# Make requests to OpenAI ChatCompletion API

input: `order_file_path`

output: `api_request_file_path`

Description
- `order_file_path`: 디스커버 정리 엑셀 파일 
- This notebook make a request to OpenAI ChatCompletion API with this data.
- Based on the Prompt Template, it writes a JSON file that contains the request data.

In [1]:
import pandas as pd
from tqdm.notebook import tqdm
import json
import re

You should specify the file path which contains your openai API key.

In [2]:
import openai
import os
with open("../specs/.openai_api_key", "r") as f:
    ss = f.readline().strip()

os.environ["OPENAI_API_KEY"] = ss

In [3]:
from openai import OpenAI
client = OpenAI()

*REMARK:*

`gpt-4o` 를 사용

In [4]:
from openai import OpenAI
client = OpenAI()

model_list = client.models.list()

In [5]:
# for x in model_list: print(x)

In [6]:
engine_id = 'gpt-4o'
N=100
dataset_size = 50000
raw_file_path = r'../local_data/(2024-04-29)구글 디스커버_데이터_수집.xlsx'
api_request_file_path = r'../local_data/api_requests_for_annotated_dataset.jsonl'

In [9]:
all_sheets = pd.read_excel(raw_file_path, sheet_name=None)
new_cols = ['url-title', 'source', 'issue-dt', 'title', 'url', 'precision', 'freshness',
       'satisfaction', 'trial-index', 'dt']

In [15]:
def make_aggregated_df(all_sheets, key):
    df = all_sheets[key]
    df.columns = new_cols
    
    # 그룹화된 'title' 열에 일련번호를 추가하여 50개씩 묶는 함수
    def add_serial_numbers_and_chunk(group):
        chunks = []
        for i in range(0, len(group), 50):
            chunk = group[i:i+50]
            numbered_titles = [f"{i+j+1}. {title}" for j, title in enumerate(chunk)]
            chunks.append('\n'.join(numbered_titles))
        return chunks

    # 새로운 DataFrame을 만들기 위한 리스트
    result = []
    
    # 그룹화하여 처리
    grouped = df.groupby('trial-index')['title']
    for trial_index, group in grouped:
        chunks = add_serial_numbers_and_chunk(group.tolist())
        for chunk in chunks:
            result.append((trial_index, chunk))
    
    # 결과 DataFrame 생성
    result_df = pd.DataFrame(result, columns=['trial-index', 'titles'])
    
    return result_df

In [16]:
make_aggregated_df(all_sheets, '김태환')

,trial-index,titles
0,0,"1. 김동현, 줄리엔강과 진짜로 '붙었다'…""마음 편해서 중간에 졸아"" 허세 \n2..."
1,1,"1. 고개숙인 버거킹 ""오해 일으켜 송구""…신제품 '뉴와퍼' 출시 \n2. &#39..."
2,1,"51. 그래도 전기차 시장은 커진다… 韓 양극재, 올해 47조원 수주 \n52. 수..."
3,2,"1. ""500㎏ 드는 것보다 공직이 더 무겁네요"" 차관으로 변신한 '로즈란' \n2..."
4,2,51. 레버쿠젠 첫 우승에… 축구 팬들이 ‘손흥민 유니폼’ 입은 까닭 \n52. “...
5,3,"1. ‘2억 혈세’ 썼는데 흉물로 전락…한강 괴물, 10년 만에 철거하기로 \n2...."
6,3,"51. ""동남아 여행 취소해야 할 판""…여름휴가 앞두고 '초비상' \n52. “제2..."
7,4,1. 이런 드라이버 레슨 없었습니다! 오른손을 놓고 쳤더니 250m 그냥 갑니다 -...
8,4,51. 03화 뼈해장국 합석은 좀… \n52. 서울대 졸업 후 ‘음악’ 하겠다며 나...
9,5,"1. “김구라도 유재석도 아니었네?!”…’유느님’ 제치고 방송 최다 출연한 MC, ..."


# Prompt Template

In [17]:
system_prompt = """뉴스 기사에 대한 카테고리를 다음과 같이 정의 했어. 잘 기억해둬.

1. 정치
   1.1. 국내 정치
       1.1.1. 청와대/행정부
       1.1.2. 국회/입법부
       1.1.3. 지방자치
       1.1.4. 선거
   1.2. 국제 정치
       1.2.1. 외교/국제관계
       1.2.2. 국제기구 (UN, NATO, EU 등)
       1.2.3. 정상회담/국제회의
   1.3. 북한
       1.3.1. 남북관계
       1.3.2. 북한 정세
       1.3.3. 북핵 문제
   1.4. 군사/안보
       1.4.1. 국방 정책
       1.4.2. 무기/방위산업
       1.4.3. 사이버 안보
       1.4.4. 테러리즘

2. 경제
   2.1. 금융
       2.1.1. 글로벌 금융시장
   2.2. 증권/주식
   2.3. 부동산
   2.4. 산업/기업
       2.4.1. 글로벌 기업 동향
   2.5. 중소기업/창업
   2.6. 경제 정책
       2.6.1. 국제 경제 정책

3. 사회
   3.1. 사건사고
       3.1.1. 국제 사건사고
   3.2. 교육
   3.3. 노동
   3.4. 환경
       3.4.1. 글로벌 환경 이슈
   3.5. 인권/복지
       3.5.1. 국제 인권

4. 생활/문화
   4.1. 건강
   4.2. 요리/맛집
   4.3. 패션/뷰티
   4.4. 여행/레저
   4.5. 자동차
   4.6. 책/문학
   4.7. 종교
   4.8. 육아/교육
   4.9. 반려동물
   4.10. 취미/DIY
   4.11. 웨딩/결혼

5. IT/과학
   5.1. 모바일
   5.2. 인터넷/SNS
   5.3. 통신/뉴미디어
   5.4. IT 기기
   5.5. 과학 일반
   5.6. 우주/항공
   5.7. 인공지능
   5.8. 블록체인/가상화폐
   5.9. 바이오테크놀로지 

6. 세계
   6.1. 아시아/호주
   6.2. 미국/중남미 
   6.3. 유럽
   6.4. 중동/아프리카
   6.5. 국제분쟁/전쟁
   6.6. 세계 일반

7. 연예
   7.1. 영화
   7.2. 방송/드라마
   7.3. 음악
   7.4. 스타/연예인
   7.5. 애니메이션/웹툰
   7.6. 셀러브리티/인플루언서

8. 스포츠
   8.1. 국내 야구
   8.2. 해외 야구
   8.3. 국내 축구 
   8.4. 해외 축구
   8.5. 골프
   8.6. 농구
   8.7. 배구
   8.8. e스포츠
   8.9. 테니스
   8.10. 격투기 (복싱, UFC 등)
   8.11. 모터스포츠 (F1, 랠리 등)
   8.12. 익스트림 스포츠

9. 오피니언
   9.1. 사설
   9.2. 칼럼
   9.3. 기자수첩
   9.4. 만평

10. 날씨
    10.1. 일기예보
    10.2. 기상이슈
    10.3. 계절/시즌 정보

11. 지역
    11.1. 수도권
    11.2. 강원/충청
    11.3. 전라/경상
    11.4. 제주

12. 행사/이벤트 
    12.1. 축제/페어
    12.2. 전시/공연
    12.3. 세미나/포럼
    12.4. 공모전/이벤트

13. 인물/인터뷰
    13.1. 정치인
    13.2. 기업인
    13.3. 학자/전문가
    13.4. 활동가/사회운동가

14. HistoHistory/역사
    14.1. 한국사
    14.2. 세계사
    14.3. 근현대사
    14.4. 문화유산/유물

### 사용자에게 제공된 개인화 뉴스 컨텐츠들의 제목을 검토한 후 핵심 개체명과 뉴스 카테고리를 할당해줘. 다음은 요구사항이야.
1. 뉴스 제목에서 핵심 개체명을 추출하자. 여러가지면 더 좋다.
2. 뉴스 제목에 대해 가장 디테일한 세부 카테고리 하나를 할당한다.
2. 라인 번호. 제목 -> (번호를 포함한) 세부 카테고리 : 개체명1 : 개체명2 : 개체명 3, ... 와 같은 형식으로 한 라인에 하니씩 반드시 표시해줘.
"""

In [18]:
user_prompt = """### 이제부터 라인마다 하나의 뉴스 제목을 나열할게 도와줘.

{0}
"""

# Asynchronous Request Preparation

In [19]:
def make_json_body(request_id, system_prompt, user_prompt):
    dic = { 
        'model':engine_id,
        'messages':[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ],
        'temperature':0.0,
        'metadata':{'request_id':request_id},
    }
    return json.dumps(dic, ensure_ascii=False)

In [22]:
with open(api_request_file_path, 'w') as f:
    cnt = 0
    sheet_names = all_sheets.keys()
    for key in sheet_names:
        df = make_aggregated_df(all_sheets, key)
        
        for i, row in df.iterrows():
            jsonl = make_json_body(
                request_id=f"{key}:{row['trial-index']}:{i}",#cnt+i,
                system_prompt=system_prompt,
                user_prompt=user_prompt.format(row.titles)
                )      
            f.write(jsonl + '\n')
            
        cnt += len(df)